# Train vs. val

# 가림 vs. 안 가림

# slow medium fast

In [ ]:
%cd /data/hyogunlee/repos/vq2d-lightning/
import json
from pathlib import Path

import numpy as np
import pandas as pd

from ltvu.metrics import get_metrics_vq2d, print_metrics_vq2d

/data/hyogunlee/repos/vq2d-lightning


/data/hyogunlee/miniconda3/envs/ltvu+/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [38]:
def calculate_diff(x1, x2, y1, y2, w1, w2, h1, h2, ori_w, ori_h, mode = 'l2_distance'):
    x1 = np.maximum(x1, 0)
    x2 = np.maximum(x2, 0)
    y1 = np.maximum(y1, 0)
    y2 = np.maximum(y2, 0)
    w1 = np.maximum(w1, 0)
    w2 = np.maximum(w2, 0)
    h1 = np.maximum(h1, 0)
    h2 = np.maximum(h2, 0)
    x_diff = (x2 - x1) / ori_w
    y_diff = (y2 - y1) / ori_h
    w_diff = (w2 - w1) / ori_w
    h_diff = (h2 - h1) / ori_h

    position_diff = (x_diff ** 2 + y_diff ** 2) ** 0.5
    size_diff = (w_diff ** 2 + h_diff ** 2) ** 0.5
    if mode == 'add':
        all_diff = position_diff + size_diff
    elif mode == 'add2':
        all_diff = (x_diff ** 2 + y_diff ** 2 + w_diff ** 2 + h_diff ** 2) ** 0.5
    elif mode == 'mul':
        all_diff = position_diff * size_diff
    elif mode == 'harmonic_mean':
        area1 = w1 * h1
        area2 = w2 * h2
        area_hm = 2 * area1 * area2 / (area1 + area2 + 1e-6)

        # Normalize changes by image dimensions if applicable
        x_diff = (x2 - x1) / (area_hm ** 0.5 + 1e-6)
        y_diff = (y2 - y1) / (area_hm ** 0.5 + 1e-6)
        w_diff = (w2 - w1) / (area_hm ** 0.5 + 1e-6)
        h_diff = (h2 - h1) / (area_hm ** 0.5 + 1e-6)
    elif mode == 'use_log':
        x_diff = np.log(1 + x2 / (x1 + 1e-6))
        y_diff = np.log(1 + y2 / (y1 + 1e-6))
        w_diff = np.log(1 + w2 / (w1 + 1e-6))
        h_diff = np.log(1 + h2 / (h1 + 1e-6))

        all_diff = np.abs(x_diff) + np.abs(y_diff) + np.abs(w_diff) + np.abs(h_diff)
    elif mode == 'l1_distance':
        w_diff = np.log(w2 / w1)
        h_diff = np.log(h2 / h1)

        all_diff = np.abs(x_diff) + np.abs(y_diff) + np.abs(w_diff) + np.abs(h_diff)
    elif mode == 'l1_distance2':
        all_diff = np.abs(x_diff) + np.abs(y_diff) + np.abs(w_diff) + np.abs(h_diff)
    elif mode == 'l2_distance':
        w_diff = np.log(w2 / w1 + 1e-6)
        h_diff = np.log(h2 / h1 + 1e-6)
        all_diff = (x_diff ** 2 + y_diff ** 2 + w_diff ** 2 + h_diff ** 2) ** 0.5


    return all_diff

In [49]:
p_ann = Path("data/vq_v2_train_anno.json")
p_pred = Path('outputs/debug/2024-11-12/35043/vq2d/predictions.json')
# metrics = get_metrics_vq2d(p_ann, p_pred)

In [50]:
all_diffs = {}
anns = json.load(p_ann.open())

quid2ann = {
    f'{ann["annotation_uid"]}_{ann["query_set"]}': ann
    for ann in anns
}

for ann in anns:
    df_gt_rt = pd.DataFrame.from_records(ann['response_track'])
    qset_uuid = f'{ann["annotation_uid"]}_{ann["query_set"]}'
    if df_gt_rt.shape[0] < 2:
        all_diffs[qset_uuid] = 0
        continue
    diffs = calculate_diff(
        df_gt_rt['x'].iloc[:-1].reset_index(drop=True),
        df_gt_rt['x'].iloc[1:].reset_index(drop=True),
        df_gt_rt['y'].iloc[:-1].reset_index(drop=True),
        df_gt_rt['y'].iloc[1:].reset_index(drop=True),
        df_gt_rt['w'].iloc[:-1].reset_index(drop=True),
        df_gt_rt['w'].iloc[1:].reset_index(drop=True),
        df_gt_rt['h'].iloc[:-1].reset_index(drop=True),
        df_gt_rt['h'].iloc[1:].reset_index(drop=True),
        ann['original_width'], ann['original_height'], mode='l2_distance')
    all_diffs[qset_uuid] = diffs.dropna().max()
    if np.isnan(all_diffs[qset_uuid]):
        print(qset_uuid)
        print(diffs)
        break


In [58]:
df_gt_rt

,fno,x,y,w,h
0,1387,1580.81,403.26,211.98,341.53
1,1388,1330.80,403.26,170.87,349.90
2,1389,1120.63,438.75,152.08,328.07
3,1390,939.50,418.25,160.62,336.62
4,1391,864.32,414.83,155.49,343.45
5,1392,879.70,325.98,143.53,324.66
6,1393,964.59,172.76,133.47,317.97
7,1394,1135.36,66.77,158.99,359.19
8,1395,1282.57,94.25,184.50,372.93
9,1396,1433.70,217.90,200.21,384.71


In [59]:
diffs

0     0.253021
1     0.175451
2     0.113609
3     0.054775
4     0.128076
5     0.166721
6     0.251025
7     0.173434
8     0.164162
9     0.136644
10    0.074934
11    0.039911
12    0.041162
13    0.041606
14    0.102496
15    0.239617
16    0.112000
dtype: float64

In [51]:
diff_thres1, diff_thres2 = np.percentile(list(all_diffs.values()), [50, 90])
diff_thres1, diff_thres2

(np.float64(0.8994432707218899), np.float64(1.8917868860078035))

In [57]:
p_tmp_ann = Path('/tmp/tmp_ann.json')
# p_tmp_pred = Path('/tmp/tmp_pred.json')
ann_subsets = {
    'slow': [],
    'medium': [],
    'fast': []
}
for qset_uuid, max_diff in all_diffs.items():
    subset_name = 'slow' if max_diff > diff_thres2 else 'medium' if max_diff > diff_thres1 else 'fast'
    ann_subsets[subset_name].append(quid2ann[qset_uuid])

for subset_name in ann_subsets:
    print(subset_name)
    p_tmp_ann.write_text(json.dumps(ann_subsets[subset_name]))
    metrics = get_metrics_vq2d(p_tmp_ann, p_pred)
    print_metrics_vq2d({'all': metrics['all']})
    print()
    print()

slow
all (0^2 - inf^2) (1361 samples / 100.0%)
VQ2D Evaluation
tAP             :  0.264
tAP  @ IoU=0.25 :  0.480
stAP            :  0.177
stAP @ IoU=0.25 :  0.433
Recovery %      : 47.030
Success         : 72.079
EgoNLQ Evaluation
R1 @ IoU=.3     : 67.230
R1 @ IoU=.5     : 61.499



medium
all (0^2 - inf^2) (5442 samples / 100.0%)
VQ2D Evaluation
tAP             :  0.270
tAP  @ IoU=0.25 :  0.495
stAP            :  0.172
stAP @ IoU=0.25 :  0.439
Recovery %      : 48.837
Success         : 72.271
EgoNLQ Evaluation
R1 @ IoU=.3     : 68.835
R1 @ IoU=.5     : 61.999



fast
all (0^2 - inf^2) (6804 samples / 100.0%)
VQ2D Evaluation
tAP             :  0.247
tAP  @ IoU=0.25 :  0.451
stAP            :  0.135
stAP @ IoU=0.25 :  0.360
Recovery %      : 46.748
Success         : 67.725
EgoNLQ Evaluation
R1 @ IoU=.3     : 65.079
R1 @ IoU=.5     : 59.259



